In [2]:
!pip install tmdbv3api

In [7]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'

In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
tmdb=TMDb()
tmdb.language='en'
movie=Movie()
num_movies=0
i=1
csv_file_path='movie_release_date_revenue_nonzero_ids.csv'
#Create a list to store movie IDs
movie_data=[]
imdb_ids = []
wikidata_ids = []
#facebook_ids = []
#instagram_ids = []
#twitter_ids = []
#Make a set of IDs to check against
written_movie_ids=set()
with open(csv_file_path,'r') as csv_file:
    csv_reader=csv.reader(csv_file)
    next(csv_reader)  #Skip the header row
    #Loop until the end of the CSV
    while True:
        try:
            row=next(csv_reader)
            movie_id=int(row[1])
            #Query movie details by ID
            ext_ids=movie.external_ids(movie_id)
            if ext_ids:
                #print(ext_ids)
                movie_data.append(movie_id)
                imdb_ids.append(ext_ids.get('imdb_id'))
                wikidata_ids.append(ext_ids.get('wikidata_id'))
                #facebook_ids.append(ext_ids.get('facebook_id'))
                #instagram_ids.append(ext_ids.get('instagram_id'))
                #twitter_ids.append(ext_ids.get('twitter_id'))
                num_movies += 1
                if num_movies%10 == 0:
                    print(num_movies)

        except StopIteration:
            break
        except Exception as e:
            #Handle any exceptions
            print(f"Error processing movie ID {movie_id}:{e}")
ext_ids_df = pd.DataFrame({'ID':movie_data, 'imdb_id': imdb_ids, 'wikidata_id': wikidata_ids})
#Create a DataFrame from the movie_data list
#df=pd.DataFrame({'ID': movie_data})
#Add an index column
#ext_ids_df.insert(0,'Index',range(1, len(ext_ids_df) + 1))
#Save
file_name='movie_external_ids.csv'
ext_ids_df.to_csv(file_name,index=False)
print(f"Operation Complete")

Pull credits:
1. go through all movies and get credits
2. get ids for each actor in the movie
3. if actor is not in actors list, add to actors list


In [ ]:
num_movies=0

csv_file_path='movie_release_date_revenue_nonzero_ids.csv'
people_info = pd.DataFrame(columns=['id', 'name', 'known_for_dept', 'popularity'])
movie_credits_df = pd.DataFrame(columns=['movie_id', 'cast_ids', 'order', 'director_ids'])
#movie_directors_df = pd.DataFrame(columns=['movie_id', 'directors_ids'])

with open(csv_file_path,'r') as csv_file:
    csv_reader=csv.reader(csv_file)
    next(csv_reader)  #Skip the header row
    #Loop until the end of the CSV
    while True:
        try:
            row=next(csv_reader)
            movie_id=int(row[1])
            credits=movie.credits(movie_id)
            if credits:
                cast = credits.get('cast')
                cast_ids = []
                order = []
                for person in cast:
                    person_id = person.get('id')
                    cast_ids.append(person_id)
                    order.append(person.get('order'))
                    # if person not in people dataframe, add them
                    if person_id not in people_info.loc[:,'id'].values:
                        person_name = person.get('name')
                        person_known = person.get('known_for_department')
                        person_pop = person.get('popularity')
                        people_info = pd.concat([people_info, pd.DataFrame({'id': [person_id], 'name': [person_name], 'known_for_dept': [person_known], 'popularity': [person_pop]})], ignore_index=True)
                crew = credits.get('crew')
                director_ids = []
                for person in crew:
                    if person.get('job') == 'Director':
                        person_id = person.get('id')
                        director_ids.append(person_id)
                        if person_id not in people_info.loc[:,'id'].values:
                            person_name = person.get('name')
                            person_known = person.get('known_for_department')
                            person_pop = person.get('popularity')
                            people_info = pd.concat([people_info, pd.DataFrame({'id': [person_id], 'name': [person_name], 'known_for_dept': [person_known], 'popularity': [person_pop]})], ignore_index=True)
                movie_credits_df = pd.concat([movie_credits_df, pd.DataFrame({'movie_id': [movie_id], 'cast_ids': [cast_ids], 'order': [order], 'director_ids': [director_ids]})], ignore_index=True)
            num_movies += 1
            if num_movies%10==0:
                print(num_movies)
        except StopIteration:
            break
        except Exception as e:
            #Handle any exceptions
            print(f"Error processing movie ID {movie_id}:{e}")

file_name='movie_credits.csv'
movie_credits_df.to_csv(file_name,index=False)

print(f"Operation Complete")

In [54]:
people_info.to_csv('people_info.csv', index=False)

In [ ]:
# LUCAS UPDATE: Get Credits for 2010, 2011-2012 data

In [10]:
num_movies=0
from tmdbv3api import TMDb, Movie
import pandas as pd
csv_file_path='clean_2010_2012_merged.csv'
people_info = pd.DataFrame(columns=['id', 'name', 'known_for_dept', 'popularity'])
movie_credits_df = pd.DataFrame(columns=['movie_id', 'cast_ids', 'order', 'director_ids'])
#movie_directors_df = pd.DataFrame(columns=['movie_id', 'directors_ids'])

with open('clean_2010_2012_merged.csv','r') as csv_file:
    csv_reader=csv.reader(csv_file)
    next(csv_reader)  #Skip the header row
    #Loop until the end of the CSV
    while True:
        try:
            row=next(csv_reader)
            movie_id=int(row[1])
            credits=movie.credits(movie_id)
            if credits:
                cast = credits.get('cast')
                cast_ids = []
                order = []
                for person in cast:
                    person_id = person.get('id')
                    cast_ids.append(person_id)
                    order.append(person.get('order'))
                    # if person not in people dataframe, add them
                    if person_id not in people_info.loc[:,'id'].values:
                        person_name = person.get('name')
                        person_known = person.get('known_for_department')
                        person_pop = person.get('popularity')
                        people_info = pd.concat([people_info, pd.DataFrame({'id': [person_id], 'name': [person_name], 'known_for_dept': [person_known], 'popularity': [person_pop]})], ignore_index=True)
                crew = credits.get('crew')
                director_ids = []
                for person in crew:
                    if person.get('job') == 'Director':
                        person_id = person.get('id')
                        director_ids.append(person_id)
                        if person_id not in people_info.loc[:,'id'].values:
                            person_name = person.get('name')
                            person_known = person.get('known_for_department')
                            person_pop = person.get('popularity')
                            people_info = pd.concat([people_info, pd.DataFrame({'id': [person_id], 'name': [person_name], 'known_for_dept': [person_known], 'popularity': [person_pop]})], ignore_index=True)
                movie_credits_df = pd.concat([movie_credits_df, pd.DataFrame({'movie_id': [movie_id], 'cast_ids': [cast_ids], 'order': [order], 'director_ids': [director_ids]})], ignore_index=True)
            num_movies += 1
            if num_movies%10==0:
                print(num_movies)
        except StopIteration:
            break
        except Exception as e:
            #Handle any exceptions
            print(f"Error processing movie ID {movie_id}:{e}")

file_name='2010_2012_movie_credits.csv'
movie_credits_df.to_csv(file_name,index=False)
print(f"Operation Complete")

FileNotFoundError: [Errno 2] No such file or directory: 'clean_2010_2012_merged.csv'